# Basic Flood Inundation Mapping using Height Above Nearest Drainage

**Authors:**  
   - Tony Castronova <acastronova@cuahsi.org>    
   - Irene Garousi-Nejad <igarousi@cuahsi.org>  
    
**Last Updated:** 05.22.2024

**Description**:  

The purpose of this Jupyter Notebook is to demonstrate the process of generating flood inundation maps (FIM) using pre-computed Height Above Nearest Drainage (HAND) raster maps. This process has been developed by the NOAA Office of Water Prediction, see the [inundation-mapping project](https://github.com/NOAA-OWP/inundation-mapping) for more information. 

There are two approaches to using the data provided by NOAA OWP to compute FIM; basic (simplified) mapping and mosaic mapping. The latter represents the state of practice in this domain, however for simplicity this notebook will demonstrate the former. After understanding the simplified approach it will be clear how to extend this work to the more complex *mosaic* methodology.

The FIM approach outlined in this notebook requires several input datasets. These can be obtained from the ESIP-hosted cloud store using the following commands (note: full instructions are provided in repository linked above):

```
aws s3 ls s3://noaa-nws-owp-fim/hand_fim/  

aws s3 sync s3://noaa-nws-owp-fim/hand_fim/outputs/fim_4_4_0_0/12090301 \
    /your_local_folder_name/12090301 
```

**Data Description**

- `rem_zeroed_masked_0.tif`: A geotif containing Height Above Nearest Drainage values.
- `gw_catchments_reaches_filtered_addedAttributes_crosswalked_0.gpkg`: Vector data and attributes for river reaches.
- `demDerived_reaches_split_filtered_addedAttributes_crosswalked_0.gpkg`: DEM-derived reach geometries that will be used for visualization.
- `hydroTable_0.csv`: A comma separated file containing pre-computed stage-discharge values for every reach in the `demDerived` dataset.

**Software Requirements**:  

The software and operating system versions used to develop this notebook are listed below. To avoid encountering issues related to version conflicts among Python packages, we recommend creating a new environment variable and installing the required packages specifically for this notebook.

> dask-core: 2023.4.0  
  geopandas: 0.12.2   
  ipyleaflet: 0.17.2  
  matplotlib: 3.7.1   
  numpy: 1.24.2  
  pandas: 2.0.0  
  scipy: 1.10.1  
  xarray: 2023.4.1  
  pynhd: 0.16.3  
  geocube: 0.5.1  
  sidecar: 0.7.0  
  rioxarray: 0.15.3  
  
**Supplementary Code**

To simplify this notebook several *helper* functions have been develop that are referenced. These functions are located in a module called `nwm_utils`.

In [ ]:
import numpy
import xarray
import rioxarray
import geopandas
import numpy as np
import pandas as pd
from typing import Dict
from pathlib import Path
from scipy import interpolate
import matplotlib.pyplot as plt
from geocube.api.core import make_geocube
from geocube.vector import vectorize

from sidecar import Sidecar
import ipyleaflet

In [ ]:
from nwm_utils.fim import map, xfim, utils

Load the `DEM Derived` reaches that have been computed by the NOAA/OWP and load them onto an interactive map. A utility library (`nwm_utils.fim.map`) is used to simplify this process.

In [ ]:
# Load DEM reaches using GeoPandas
geodf_reaches = geopandas.read_file(Path('./sample-data/demDerived_reaches_split_filtered_addedAttributes_crosswalked_0.gpkg'))
transformed_reaches = geodf_reaches.to_crs(epsg='4326')

In [ ]:
# Create an interactive map using SideCar
sc = map.SideCarMap(gdf=transformed_reaches)
sc.display_map()

Now we need to select an area of interest on the map. Once a reach has been selected, the following cell can be executed to get its metadata.

In [ ]:
nhd_feature_id = sc.selected().feature_id
sc.selected()

Next we need to define the streamflow to use to compute the flood inundation. This can be a hypothetical streamflow for now, but lets look at the stage-discharge relationships defined in the `hydroTable.csv` file to choose a reasonable value. Note, this can be easily replaced with logic to compute streamflow based on historical flows.

The reach that we selected in the map is subdivided into multiple segments, each having a unique stage-discharge relationship. Let's use a utility function to plot these relationships.

In [ ]:
fig, ax = plt.subplots()


# collect the hydroids for all the segments within the selected reach.
d = utils.get_hydroids_for_reach(nhd_feature_id, hydrotable=Path('sample-data/hydroTable_0.csv'))

# get unique combos of HydroID and NextDownID
hydro_ids = np.unique(d.HydroID)
hydro_ids

# read the rating curve data
hydrotable = pd.read_csv('sample-data/hydroTable_0.csv',
                         usecols=["HydroID",
                                  "NextDownID",
                                  "feature_id",
                                  "stage",
                                  "discharge_cms"])

# collect rating curve data for each hydroid and plot
for hydro_id in hydro_ids:
    dat = hydrotable.loc[hydrotable.HydroID == hydro_id, ["stage", "discharge_cms"]]
    dat.plot(x='discharge_cms', y='stage', label=hydro_id, ax=ax)

ax.set_xlabel('River Discharge (cms)')
ax.set_ylabel('River Stage (m)')
plt.title(f'Rating Curves for NHD Reach: {nhd_feature_id}')
plt.grid(True)


In [ ]:
# hypothetical streamflow (cms).
# This can be replaced with a flow value gathered by a source of your choice
cms = 5000

Use the streamflow defined above to interpolate river stage from a rating curve for all hydroids that exist within this NHD+ reach. 

In [ ]:
stage_dict = xfim.get_stage_for_all_hydroids_in_reach(int(nhd_feature_id), cms, hydrotable=Path('sample-data/hydroTable_0.csv'))
stage_dict

Load the precomputed HAND raster.

In [ ]:
xds = rioxarray.open_rasterio(Path('sample-data/rem_zeroed_masked_0.tif'),
                              masked=True).squeeze().drop_vars('band').to_dataset(name='hand')
xds

Make a copy of the 'hand' variable to the 'stage' variable. This will be used to compute the flood inundation map later on. The dataset will now have both `hand` and `stage` variables for all (x,y) locations.

In [ ]:
xds['stage'] = xds.hand.copy(deep=True)
xds

Read watershed geometries and set stage values from the `stage_dict` defined above, and remove all other geometries. The end product will be a geodataframe containing the geometries for each `hydroid` in our area of interest and their stage values.

In [ ]:
geodf = geopandas.read_file(Path('sample-data/gw_catchments_reaches_filtered_addedAttributes_crosswalked_0.gpkg'))

# loop over each reach and set the corresponding stage
# in the geopandas object
for hydroid, stage in stage_dict.items():
    geodf.loc[geodf.HydroID==hydroid, 'stage'] = stage

# remove all nan values to make our dataset smaller
geodf_filtered = geodf[geodf.stage.notnull()]

# isolate only reaches we're interested in to make our dataset smaller
geodf_reaches_filtered = geodf_reaches[geodf_reaches.feature_id == nhd_feature_id]

geodf_filtered

Visualize the area that we're working with.

In [ ]:
figure, axes = plt.subplots(1, 2, figsize=(10, 10))

geodf.plot(facecolor='None', edgecolor='lightgrey', ax=axes[0]);
geodf_filtered.plot(facecolor='green', ax=axes[0]);

geodf_filtered.plot(facecolor='None', ax=axes[1]);
geodf_reaches_filtered.plot(ax=axes[1]);

Add the geometries for our area of interest to the Xarray Dataset containing `hand` and `stage` variables using a GeoCube. This will enable us to set grid values corresponding with catchment areas.

In [ ]:
# create a grid for the geocube
out_grid = make_geocube(
    vector_data=geodf,
    measurements=['HydroID'],
    like=xds # ensure the data are on the same grid
)

# add stage and hydroID to the HAND raster
ds = xds.assign_coords( hydroid = (['y', 'x'], out_grid.HydroID.data) )

# drop everything except the HydroIDs that we're interested in
ds = ds.where(ds.hydroid.isin(geodf_filtered.HydroID), drop=True)
ds

Update the stage values in the DataSet where specific hydroid's exist.

In [ ]:
for idx, row in geodf_filtered.iterrows():
    print(f'{row.HydroID} -> {row.stage}' )
    ds['stage'] = xarray.where(ds.hydroid == row.HydroID, row.stage, ds.stage)

Preview the `stage` data we set as well as the original `hand` data. 

In [ ]:
figure, axes = plt.subplots(1, 2, figsize=(10,5))

# plot stage
ds.stage.plot(ax=axes[0])
axes[0].set_title('Stage');
axes[0].tick_params(axis='x', labelrotation=45)

# plot hand
ds.hand.plot(ax=axes[1])
axes[1].set_title('HAND');
axes[1].tick_params(axis='x', labelrotation=45)

figure.tight_layout()


Compute FIM by subtracting `hand` from `stage`. Everything that is negative should be set to zero. Cells that have a value greater than zero indicate areas in which flooding occurs. Create a new variable in our dataset (called `fim`) to store this result.

In [ ]:
ds['fim'] = ds.stage - ds.hand
ds['fim'] = xarray.where(ds.fim >= 0.00001, ds.fim, numpy.nan)

In [ ]:
figure, axes = plt.subplots(1, 2, figsize=(12,5))

geodf.loc[geodf.feature_id==nhd_feature_id].plot(ax=axes[0], edgecolor='grey', facecolor='None');
ds.fim.plot(cbar_kwargs={'label':'depth [meters]'}, ax=axes[0]);
axes[0].set_title(f'FIM for NHD {nhd_feature_id} at {cms} cms');


xarray.plot.contourf(ds.fim, levels=4, cmap='Blues', cbar_kwargs={'label':'Flood Risk (light = low risk)'}, ax=axes[1]);
geodf.loc[geodf.feature_id==nhd_feature_id].plot(ax=axes[1], edgecolor='grey', facecolor='None');
axes[1].set_title(f'FIM Contours for NHD {nhd_feature_id} at {cms} cms');


In [ ]:
# drop nans
da = ds.fim
df = vectorize(da)
df = df[df.fim.notna()]

In [ ]:
transformed_fim = df.to_crs(epsg='4326')
poly = transformed_fim.dissolve()
poly.fim = 1

In [ ]:
wlayer = ipyleaflet.WKTLayer(
            wkt_string=poly.iloc[0].geometry.wkt,
            style={'color': 'red', 'opacity':.5, 'weight':1.9,})
sc.map.add(wlayer);